In [1]:
import pandas as pd
import numpy as np
import re

%matplotlib inline

In [2]:
subway = pd.read_csv('subway_stations.csv')

In [3]:
def new_name(s):
    a = re.sub(r"(?<=\d)(st|nd|rd|th)\b", '', s)
    a = re.sub(r"(?<=\S) Ave", " Av", a)
    a = re.sub(r"Ave (?=\w)", "Avenue ", a)
    a = re.sub(r" - ", "-", a)
    a = re.sub("Pky", "Pkwy", a)

    
    return a

subway.NAME = subway.NAME.apply(lambda x: new_name(x))

In [4]:
subway.head(5)

,URL,OBJECTID,NAME,the_geom,LINE,NOTES
0,http://web.mta.info/nyct/service/,1,Astor Pl,POINT (-73.99106999861966 40.73005400028978),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
1,http://web.mta.info/nyct/service/,2,Canal St,POINT (-74.00019299927328 40.71880300107709),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
2,http://web.mta.info/nyct/service/,3,50 St,POINT (-73.98384899986625 40.76172799961419),1-2,"1-all times, 2-nights"
3,http://web.mta.info/nyct/service/,4,Bergen St,POINT (-73.97499915116808 40.68086213682956),2-3-4,"4-nights, 3-all other times, 2-all times"
4,http://web.mta.info/nyct/service/,5,Pennsylvania Av,POINT (-73.89488591154061 40.66471445143568),3-4,"4-nights, 3-all other times"


In [5]:
def split(s):
    return s.split('-')

In [6]:
subway.LINE = list(map(split, subway.LINE))
subway.head(5)

,URL,OBJECTID,NAME,the_geom,LINE,NOTES
0,http://web.mta.info/nyct/service/,1,Astor Pl,POINT (-73.99106999861966 40.73005400028978),"[4, 6, 6 Express]","4 nights, 6-all times, 6 Express-weekdays AM s..."
1,http://web.mta.info/nyct/service/,2,Canal St,POINT (-74.00019299927328 40.71880300107709),"[4, 6, 6 Express]","4 nights, 6-all times, 6 Express-weekdays AM s..."
2,http://web.mta.info/nyct/service/,3,50 St,POINT (-73.98384899986625 40.76172799961419),"[1, 2]","1-all times, 2-nights"
3,http://web.mta.info/nyct/service/,4,Bergen St,POINT (-73.97499915116808 40.68086213682956),"[2, 3, 4]","4-nights, 3-all other times, 2-all times"
4,http://web.mta.info/nyct/service/,5,Pennsylvania Av,POINT (-73.89488591154061 40.66471445143568),"[3, 4]","4-nights, 3-all other times"


In [7]:
import itertools
trains = list(itertools.chain.from_iterable(subway.LINE))

In [8]:
lines = list(set(trains))

In [9]:
num_s = len(subway)

In [10]:
st_matrix = pd.DataFrame(data = np.zeros([num_s,len(lines)]), columns = lines)
a = subway.LINE[0]

In [11]:
st_matrix.iloc[0][(a == lines)]

0.0

In [12]:
def rp(a): 
    bl = []
    for i, l in enumerate(lines):
        if l in a:
            bl.append(1)
        else:
            bl.append(0)
    return bl
m1 = list(map(rp, subway.LINE))
subway[lines] = pd.DataFrame(m1)

In [13]:
subway.head(5)

,URL,OBJECTID,NAME,the_geom,LINE,NOTES,A,N,J,E,...,D,7 Express,Q,C,M,4,W,6,2,3
0,http://web.mta.info/nyct/service/,1,Astor Pl,POINT (-73.99106999861966 40.73005400028978),"[4, 6, 6 Express]","4 nights, 6-all times, 6 Express-weekdays AM s...",0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1,http://web.mta.info/nyct/service/,2,Canal St,POINT (-74.00019299927328 40.71880300107709),"[4, 6, 6 Express]","4 nights, 6-all times, 6 Express-weekdays AM s...",0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
2,http://web.mta.info/nyct/service/,3,50 St,POINT (-73.98384899986625 40.76172799961419),"[1, 2]","1-all times, 2-nights",0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,http://web.mta.info/nyct/service/,4,Bergen St,POINT (-73.97499915116808 40.68086213682956),"[2, 3, 4]","4-nights, 3-all other times, 2-all times",0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
4,http://web.mta.info/nyct/service/,5,Pennsylvania Av,POINT (-73.89488591154061 40.66471445143568),"[3, 4]","4-nights, 3-all other times",0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [14]:
def lang(s):
    s = s.split(' ')
    lang = s[2].strip(')')
    return float(lang)
def long(s):
    s = s.split(' ')
    long = s[1].strip('(')
    return float(long)

In [15]:
subway['lang'] = list(map(lang, subway['the_geom']))
subway['long'] = list(map(long, subway['the_geom']))

In [16]:
subway.drop(['URL','OBJECTID', 'the_geom', 'NOTES'], axis = 1, inplace= True)

In [17]:
subway.head()

,NAME,LINE,A,N,J,E,F,L,Z,6 Express,...,Q,C,M,4,W,6,2,3,lang,long
0,Astor Pl,"[4, 6, 6 Express]",0,0,0,0,0,0,0,1,...,0,0,0,1,0,1,0,0,40.730054,-73.991070
1,Canal St,"[4, 6, 6 Express]",0,0,0,0,0,0,0,1,...,0,0,0,1,0,1,0,0,40.718803,-74.000193
2,50 St,"[1, 2]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,40.761728,-73.983849
3,Bergen St,"[2, 3, 4]",0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,1,40.680862,-73.974999
4,Pennsylvania Av,"[3, 4]",0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,40.664714,-73.894886


In [18]:
a = str('Line_') + lines[1].replace(' ', '_')
exec("%s = %d" % (a,2))

In [19]:
d = {}
lines_name = []
for line_name in lines:
    col_name = subway.columns
    var_name = line_name.replace(' ', '_') + str('_LINE') 
    sub_df = subway[subway[line_name] == 1]
    sub_df =  sub_df[['NAME','LINE','lang', 'long']]
    
    
    sub_df['top_to_bottom'] = np.zeros((len(sub_df),1))
    sub_df['reason1'] = np.zeros((len(sub_df),1))
    sub_df['bottom_to_top'] = np.zeros((len(sub_df),1))
    sub_df['reason2'] = np.zeros((len(sub_df),1))
    
    sub_df[['top_to_bottom', 'reason1', 'bottom_to_top', 'reason2']] =  sub_df[['top_to_bottom', 'reason1', 'bottom_to_top', 'reason2']].astype(str)
    
    sub_df['name_split'] = sub_df.NAME.apply(lambda x: x.replace('-', ' ').split())
    
    d[var_name] = sub_df
    lines_name.append(var_name)

#### Re-arrange subway stations

In [20]:
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
import numpy as np
from scipy.spatial.distance import cdist

In [22]:
def find_orgin(ls):
    pos = 0
    ls_path = ls.copy()
    for i in range(len(ls) - 1):
        point = ls[pos].copy()
        ls_path[pos] = np.array([0,0])
        pos = np.argmin(cdist(ls[pos].reshape(1,-1), ls_path))
    return pos

In [23]:
def find_path(ls, start):
    pos = start
    path = [pos]

    ls_path = ls.copy()
    for i in range(len(ls) - 1):
        point = ls[pos].copy()
        ls_path[pos] = np.array([0,0])
        pos = np.argmin(cdist(ls[pos].reshape(1,-1), ls_path))
        path.append(pos)
    return path

In [24]:
def rearrange(df):
    ls = []
    test = df.copy()
    for i in range(len(test)):
        a = [test.iloc[i]['long'], test.iloc[i]['lang']]
        ls.append(a)
    ls = np.array(ls)
    
    start = find_orgin(ls)
    path = find_path(ls, start)
    result = test.iloc[path]
    
    result.reset_index(inplace = True)
    result.drop('index', axis = 1, inplace = True)
    return result

In [25]:
new_d = {}
for l in lines_name:
    vname = l
    copy = d[l].copy()
    df = rearrange(copy)
    new_d[vname] = df

In [26]:
for l in lines_name:
    filename = 'train_datasets/' + l + '.csv'
    new_d[l].to_csv(filename, index = False)

In [27]:
#np.save('stations_dic.npy', new_d)

In [28]:
new_d['F_LINE'].head()

,NAME,LINE,lang,long,top_to_bottom,reason1,bottom_to_top,reason2,name_split
0,Jamaica-179 St,[F],40.712646,-73.783817,0.0,0.0,0.0,0.0,"[Jamaica, 179, St]"
1,169 St,[F],40.710518,-73.793474,0.0,0.0,0.0,0.0,"[169, St]"
2,Parsons Blvd,[F],40.707572,-73.803289,0.0,0.0,0.0,0.0,"[Parsons, Blvd]"
3,Sutphin Blvd,[F],40.705418,-73.810833,0.0,0.0,0.0,0.0,"[Sutphin, Blvd]"
4,Briarwood-Van Wyck Blvd,"[E, F]",40.709162,-73.820693,0.0,0.0,0.0,0.0,"[Briarwood, Van, Wyck, Blvd]"


In [29]:
new_d['F_LINE']

,NAME,LINE,lang,long,top_to_bottom,reason1,bottom_to_top,reason2,name_split
0,Jamaica-179 St,[F],40.712646,-73.783817,0.0,0.0,0.0,0.0,"[Jamaica, 179, St]"
1,169 St,[F],40.710518,-73.793474,0.0,0.0,0.0,0.0,"[169, St]"
2,Parsons Blvd,[F],40.707572,-73.803289,0.0,0.0,0.0,0.0,"[Parsons, Blvd]"
3,Sutphin Blvd,[F],40.705418,-73.810833,0.0,0.0,0.0,0.0,"[Sutphin, Blvd]"
4,Briarwood-Van Wyck Blvd,"[E, F]",40.709162,-73.820693,0.0,0.0,0.0,0.0,"[Briarwood, Van, Wyck, Blvd]"
5,Kew Gardens-Union Tpke,"[E, F]",40.714035,-73.830370,0.0,0.0,0.0,0.0,"[Kew, Gardens, Union, Tpke]"
6,75 Av,"[E, F]",40.718045,-73.836924,0.0,0.0,0.0,0.0,"[75, Av]"
7,Forest Hills-71 Av,"[E, F, M, R]",40.721594,-73.844517,0.0,0.0,0.0,0.0,"[Forest, Hills, 71, Av]"
8,Jackson Hts-Roosevelt Av,"[E, F, M, R]",40.746540,-73.891299,0.0,0.0,0.0,0.0,"[Jackson, Hts, Roosevelt, Av]"
9,21 St-Queensbridge,[F],40.753739,-73.941938,0.0,0.0,0.0,0.0,"[21, St, Queensbridge]"


In [31]:
lines_name

['A_LINE',
 'N_LINE',
 'J_LINE',
 'E_LINE',
 'F_LINE',
 'L_LINE',
 'Z_LINE',
 '6_Express_LINE',
 '1_LINE',
 '5_LINE',
 '7_LINE',
 'R_LINE',
 'B_LINE',
 'G_LINE',
 'S_LINE',
 'D_LINE',
 '7_Express_LINE',
 'Q_LINE',
 'C_LINE',
 'M_LINE',
 '4_LINE',
 'W_LINE',
 '6_LINE',
 '2_LINE',
 '3_LINE']